In [24]:
import csv
import os
import pandas as pd

folders = os.listdir('dataset_caed')
dataset = []

category_map = {}

for i, folder in enumerate(folders):
    category_map[folder] = bin(i)

print(category_map)

for folder in folders:
    files = os.listdir(f'dataset_caed/{folder}')
    #print(folder)
    for file in files:
        #print(file)
        with open(f'dataset_caed/{folder}/{file}', 'r', encoding='utf-8-sig') as f:
           dataset.append({"category": category_map[folder], "name": file, "text": f.read()})

df = pd.DataFrame(dataset)
df

{'2_Ensino_Fundamental_II': '0b0', '3_Ensino_Medio': '0b1', '1_Ensino_Fundamental_I': '0b10', '4_Ensino_Superior': '0b11'}


,category,name,text
0,0b0,53_2746.txt,"Como funciona, Origem das coisas. “É tarde! É ..."
1,0b0,65_2775.txt,A última rodada do primeiro turno do Campeonat...
2,0b0,204_5035.txt,Em relação ao adolescente (maior de doze anos)...
3,0b0,111_2840.txt,Curiosidades. Muitas pessoas dizem que o riso ...
4,0b0,306_5137.txt,"O trabalho infantil, principalmente em carvoa..."
...,...,...,...
2071,0b11,377_1409.txt,Mecânica dos fluidos/Tensão superficial\nA pr...
2072,0b11,451_1483.txt,Otimização/Método de gradientes conjugados\nE...
2073,0b11,292_1321.txt,Mecânica dos fluidos/Exercícios resolvidos/A2\...
2074,0b11,263_1292.txt,Logística/Movimentação de materiais/Equipament...
